langsmith digunakan untuk tracking block chain yang digunakan pada langchain

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")

1. Model

In [4]:
from langchain_community.chat_models import ChatOllama

/Users/a/Programming/Langchain-Project/my-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm_model = ChatOllama(
    model='mistral-openorca:7b-q4_0',
    temperature=0
)

/var/folders/05/16tm42991snf6c27rs7nssdh0000gn/T/ipykernel_13917/85654660.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm_model = ChatOllama(


2. prompt template

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [7]:
prompt = ChatPromptTemplate.from_template(
    """Kamu adalah asisten chatbot yang membantu. 
    Tugasmu adalah menjawab pertanyaan berikut secara langsung dalam Bahasa Indonesia. 
    Meskipun pertanyaan diberikan dalam Bahasa Inggris atau bahasa lain, kamu WAJIB menjawabnya dalam Bahasa Indonesia.
    
    Pertanyaan: {input}
    Jawaban:"""
)

In [14]:
interface_user = "bisa jelaskan apa itu jokowi dodo"

In [15]:
chain = prompt | llm_model | StrOutputParser()

In [16]:
respone = chain.invoke({"input": interface_user})
print(respone)

 Jokowi Dodo adalah sebuah film animasi yang merupakan adaptasi dari cerita tradisional Indonesia tentang perjuangan seorang burung yang bernama Joko, atau lebih dikenal dengan nama "Jokowi" dalam bahasa Indonesia. Di dalam cerita ini, Jokowi berjuang untuk mencari kembali teman-teman yang tersingir oleh satu burung pemburu yang bernama Dodo. Film ini didirikan untuk memberikan pesan tentang keberani, semangat, dan kesahastraan dalam mengatasi permasalahan yang disadari.


3. Embedding model -> digunakan untuk mengubah external data ke dalam bentuk vector 

In [7]:
# memanggil Indobert dari transformer
from transformers import BertTokenizer, AutoModel

tokenizer = BertTokenizer.from_pretrained("Indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [8]:
# membuat class model embdding
from typing import List 
from langchain_core.embeddings import Embeddings
import torch

class IndoBertEmbeddings(Embeddings):
    def __init__(self, model_name="indobenchmark/indobert-base-p1"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()


    def _generate_embedding(self, text: str) -> List[float]:
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        with torch.no_grad():
            outputs = self.model(**inputs)

        # polling token menjadi satu vector kalimat
        token_embeddings = outputs.last_hidden_state

        # melakukan mean polling
        sentence_embeddings = token_embeddings.mean(dim=1)

        # konversi ke list python
        return sentence_embeddings.squeeze().tolist()
    

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._generate_embedding(text) for text in texts]
    

    # metode untuk pencarian query pada chroma 
    def embed_query(self, text: str) -> List[float]:
        return self._generate_embedding(text)


In [9]:
embeddings = IndoBertEmbeddings()

4. Vector store (kita akan menggunakan chromadb yang sebelumnya telah disetup)

In [10]:
from langchain_chroma import Chroma

In [11]:
vectore_store = Chroma(
    collection_name="chroma-with-langchain",
    embedding_function=embeddings,
    persist_directory="../Langchain-Project/notebooks/chroma_langchain_db"
)

5. Indexing (Load -> split -> store)

![image.png](https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=1838328a870c7353c42bf1cc2290a779)

In [1]:
# load documents dan kita ubah kedalam bentuk document standart langchain yaitu document object 
import bs4
from langchain_community.document_loaders import WebBaseLoader

/Users/a/Programming/Langchain-Project/my-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post_header", "post-content"))
# loader = WebBaseLoader(
#     web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),  # data diambil dari blognya lilianweng
#     bs_kwargs={"parse_only": bs4_strainer}
# )

# ngambil data dari web sttnf
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post_header", "post-content"))
loader = WebBaseLoader(
    web_path=("https://nurulfikri.ac.id/",),  # data diambil dari blognya lilianweng
    bs_kwargs={"parse_only": bs4_strainer}
)



In [ ]:
docs = loader.load()

In [28]:
assert len(docs) == 1
print(f"Total character: {len(docs[0].page_content)}")

Total character: 42964


In [31]:
print(docs[0].page_content[:600])


      LLM Powered Autonomous Agents
    Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large t


Selanjutnya adalah splitting document
Karena document yang telah di load itu sangat panjang, sekita 42K, model LLM akan kesulitan menjadikan ini sebagai input. Maka dari itu kita bagi ini menjadi beberapa chuncks atau potongan

In [32]:
# splitter 

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, #chunk size (characters)
    chunk_overlap=200, # overlap (Characters)
    add_start_index=True,
)

In [34]:
all_splits = text_splitter.split_documents(docs)

In [35]:
print(f"split blog post into {len(all_splits)} sub-documents.")

split blog post into 63 sub-documents.


In [40]:
print(all_splits[0])

page_content='LLM Powered Autonomous Agents
    Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 7}


In [42]:
# storing document kedalam chromaDB
document_ids = vectore_store.add_documents(documents=all_splits)

In [43]:
print(document_ids[:3])

['7f0dba64-9df4-4f98-b236-e65a05ee59cb', '65d37f22-25e1-4497-8b0f-9af2842c51e2', '0ffa8ee9-1029-429a-ac30-0848f626c9d7']


In [66]:
query = "What is task decomposition?"

In [67]:
testing_ = vectore_store.similarity_search(query=query, k=3)

In [68]:
for doc in testing_:
    print(doc.page_content)

You always add a comment briefly describing the purpose of the function definition.
You try to add comments explaining very complex bits of logic.
You always follow the best practices for the requested languages in terms of describing the code written as a defined
package/project.
Python toolbelt preferences:
Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)

The system comprises of 4 stages:
(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.
Instruction:
(4) Response generation: LLM receives the execution results and provides summarized results to users.
To put HuggingGPT into real world usage, a couple challenges need to solve: (1) Efficiency improvement is needed as both LLM inference rounds and interactions with other models slow down the process; (2) It